# Character level language model - Dinosaurus Island

6천 5백만년 전 생존하던 공룡이 있는 섬에 오신것을 환영합니다!!!

생물학의 선구적인 연구자들이 새로운 종류의 공룡을 만들어 지구상에 살아가게 만들었습니다. 그리고 당신의 특별 임무는 공룡들에게 이름을 대는 것이다.

만약 공룡의 이름이 마음에 안들면 공격적으로 변할 수 있으니 조심히 선택하세요~


<table>
<td>
<img src="images/dino.jpg" style="width:250;height:300px;">

</td>

</table>

운좋게도 딥러닝 기술을 배웠고 그 기술로 공룡의 공격성을 자극시키지 않도록 해봅시다.

당신의 조수는 찾을 수 있는 모든 종류의 공룡의 이름을 수집해왔습니다. 그리고 [dataset](dinos.txt)에 저장해 뒀습니다.


새로운 공룡의 이름을 만들기 위해 언어 모델을 구축할겁니다.

언어 모델은 다양한 분야의 이름을 학습하고 랜덤하게 새로운 이름을 만드는 역할을 합니다.

모델이 공룡의 온순한 상태로 유지해 우리 팀원이 안전하길 바래봅니다.


이번 과제를 끝마치면 얻을 수 있는 정보:

- RNN에 입력하기 위해 전처리한 텍스트 데이터 저장하는 방법
- 모든 RNN-cell의 결과물인 데이터를 함성하는 방법 <!-- - How to synthesize data, by sampling predictions at each time step and passing it to the next RNN-cell unit-->

- 글자 단위 RNN 모델을 구축하는 방법

- <!--Why clipping the gradients is important-->

일단 필요한 패키지를 호출합니다. 그 중 `rnn_utils`는 과제에서 제공하는 패키지입니다.

이전 과제에서 구현했던 `rnn_forward`와 `rnn_backward`를 사용할 수 있습니다.

*gradient를 기울기, 변화량, 경사도*로 표현합니다.

In [ ]:
import numpy as np
from utils import *
import random
import pprint

## 1 - Problem Statement

### 1.1 - Dataset and Preprocessing

아래 코드를 실행시켜 공룔이름이 저장된 데이터를 읽고 모든 이름 갯수를 확인해보세요~

In [ ]:
data  = open('dinos.txt', 'r').read()
data  = data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

* 이름에 사용된 알파벳은 26개입니다. 나머지 1은 "\n"입니다. ("\n"은 줄바꿈 때문에 생기는 값입니다.)

* 이번 과제에서 "\n"은 강의에서 언급했던 `<EOS>`와 비슷한 역할을 합니다. (End of Sentence)
    - 여기서 "\n"는 문장의 끝을 의미하는게 아닌 공룡 이름의 끝을 의미합니다.
* `char_to_ix`: 하단 셀은 각 알파벳과 인덱스(0-26) 값은 매핑해 dict 자료형으로 저장한 값입니다.
* `ix_to_char`: 하단 셀에서 두번째는 인덱스(0-26)와 각 알파벳 값을 매핑해 dict 자료형으로 저장한 값입니다.
    -  이것은 나중에 softmax 분류 층에서 가장 높은 확률의 값인 인덱스와 문자를 매핑으로 어떤 알파벳인지 알려줍니다.

In [ ]:
chars = sorted(chars)
print(chars)

In [ ]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(ix_to_char)

### 1.2 - Overview of the model

만들 모델은 다음 구조와 같습니다: 

- 매개변수 초기화
- 반복문을 통해 최적화
    - 비용함수를 계산하는 순전파 진행
    - 역전파 과정: 손실함수에 대해 경사도를 구하는 작업
    - 기울기 발산을 피하기 위해 기울기값에 clip 적용 (clip: min과 max 값을 하이퍼파라미터로 받고 해당 값을 넘지 않도록 설정하는 것)
    - 구한 기울기를 통해 매개변수들 업데이트
- 학습된 매개변수 반환
    
<img src="images/rnn.png" style="width:450;height:300px;">
<caption><center> **Figure 1**: 이전 과정에서 만든 RNN과 비슷한 구조입니다.  </center></caption>

* 각 시점에서 현재 입력값과 예전 정보를 통해 다음에 올 알파벳을 예측합니다.

* 데이터 $\mathbf{X} = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$ 는 문자를 list자료형으로  자장한 학습데이터 입니다. 
* $\mathbf{Y} = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$ 또한 같은 알파벳을 list 자료형으로 저장한 예측 값입니다. 다른점은 한 시점이 미뤄져있다는 것?

* 모든 시점 $t$로 비교해서 봅시다. $y^{\langle t \rangle} = x^{\langle t+1 \rangle}$.  $t$ 시점 예측값은 $t + 1$시점 입력값과 같습니다.

## 2 - Building blocks of the model

이 부분에서는 모델에서 가장 중요한 2 부분을 구현할겁니다.
- Gradient clipping: 기울기가 발산하지 않도록 하는 장치
- Sampling: 알파벳을 생성하는 방법

이 두 기능을 나중에 모델에 사용할겁니다.

### 2.1 - Clipping the gradients in the optimization loop

이번 섹션에서는 반복문 중 최적화하는 과정에서 `clip`을 사용할겁니다.

#### Exploding gradients
* 기울기가 매우 커지는 것을 "기울기 발산"이라고 합니다 
* 기울기 발산은 학습을 더욱 어렵게 만듭니다. 기울기 값에 따라 매개변수가 변하는데 최적값과 다르게 업데이트 되기 때문입니다.

반복 구조의 구성:
* 순전파
* 비용함수 계산
* 역전파
* 매개변수 업데이트

매개변수를 업데이트하기 전에 기울기가 발산하는 것을 막기 위해 기울기에 clipping을 적용합니다.

#### gradient clipping
`clip`기능는 dict 자료형으로 저장된 변화량을 받고 clip 적용된 변화량을 출력합니다.

* clip에는 다양한 방법이 있습니다.

* 이번에는 간단한 clip 방법을 사용할겁니다. 이 방법은 모든 기울기 값이 [-N, N]사이에 있도록 만들어줍니다.
* 예를 들어, 만약 N=10 이라면
    - 범위는 [-10, 10]
    - 만약 10보다 큰 값이 있다면 자동으로 10값으로 변경됩니다.
    - -10 보다 작은 값이 있다면 자동으로 -10값으로 변경됩니다.
    - -10 과 10 사이 값들은 해당 값 그대로 유지됩니다.

<img src="images/clip.png" style="width:400;height:150px;">
<caption><center> **Figure 2**: Gradient clipping 유무에 따른 경사하강법 과정을 시각화한겁니다. </center></caption>

**Exercise**: 
아래 함수는 clipped 기울기를 반환합니다.
* 해당 함수는 임계치로 최댓값을 받습니다. (clipped gradients를 위해)
* 공식 문서를 참조해보세요 [numpy.clip](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.clip.html). 
    - clip 기능의 하이퍼 파라미터 "`out = ...`"이 필요할 수 있습니다.
    - "`out`"을 사용한다면 "out"에 위치한 곳에 업데이트된 gradient가 저장됩니다.
    - 만약 "`out`"을 사용하지 않는다면 자동으로 "gradient"가 업데이트 되지 않습니다.

In [ ]:
### GRADED FUNCTION: clip
def clip(gradients, maxValue):
    '''
    Clips the gradients' values between minimum and maximum.
    
    Arguments:
    gradients -- a dictionary containing the gradients "dWaa", "dWax", "dWya", "db", "dby"
    maxValue -- everything above this number is set to this number, and everything less than -maxValue is set to -maxValue
    
    Returns: 
    gradients -- a dictionary with the clipped gradients.
    ''' 
    # dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    # clip to mitigate exploding gradients, loop over [dWax, dWaa, dWya, db, dby]. (≈2 lines)
    for key, gradient in gradients.items():
        gradients[key] = np.clip( gradients[key], -maxValue, maxValue )        
    
    # gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    return gradients

In [ ]:
# Test with a maxvalue of 10
maxValue = 10
np.random.seed(3)
dWax = np.random.randn(5,3)*10
dWaa = np.random.randn(5,5)*10
dWya = np.random.randn(2,5)*10
db   = np.random.randn(5,1)*10
dby  = np.random.randn(2,1)*10
gradients = {"dWax": dWax, "dWaa": dWaa, "dWya": dWya, "db": db, "dby": dby}
gradients = clip(gradients, maxValue)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])

** Expected output:**

```Python
gradients["dWaa"][1][2] = 10.0
gradients["dWax"][3][1] = -10.0
gradients["dWya"][1][2] = 0.29713815361
gradients["db"][4] = [ 10.]
gradients["dby"][1] = [ 8.45833407]
```

In [ ]:
# Test with a maxValue of 5
maxValue = 5
np.random.seed(3)
dWax = np.random.randn(5,3)*10
dWaa = np.random.randn(5,5)*10
dWya = np.random.randn(2,5)*10
db = np.random.randn(5,1)*10
dby = np.random.randn(2,1)*10
gradients = {"dWax": dWax, "dWaa": dWaa, "dWya": dWya, "db": db, "dby": dby}
gradients = clip(gradients, maxValue)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])

** Expected Output: **
```Python
gradients["dWaa"][1][2] = 5.0
gradients["dWax"][3][1] = -5.0
gradients["dWya"][1][2] = 0.29713815361
gradients["db"][4] = [ 5.]
gradients["dby"][1] = [ 5.]
```

### 2.2 - Sampling

이제 당신의 모델이 학습됐다고 가정하고 새로운 알파벳을 생성해봅시다. 생성되는 과정은 하단 이미지로 설명합니다.

<img src="images/dinos3.png" style="width:500;height:300px;">
<caption><center> **Figure 3**: 해당 사진에서 모델은 학습된 모델을 가정합니다. 첫 시점에서 $x^{\langle 1\rangle} = \vec{0}$를 할당하고 한시점에 한 알파벳을 생성합니다.</center></caption>

**Exercise**:
알파벳을 생성하는 `sample`을 구현하기 위해서는 다음 4단계를 거쳐야합니다.

- **Step 1**: 입력값으로 0행렬$x^{\langle 1 \rangle} = \vec{0}$을 사용합니다.
    - 새로 알파벳을 생성하기전에 하는 기본적인 규칙입니다. 또한 $a$도 $a^{\langle 0 \rangle} = \vec{0}$로 설정합니다.

- **Step 2**: 첫번째 단계를 거쳐 $a^{\langle 1 \rangle}$와 $\hat{y}^{\langle 1 \rangle}$를 계산합니다. 수식은 다음과 같습니다.

hidden state:  
$$ a^{\langle t+1 \rangle} = \tanh(W_{ax}  x^{\langle t+1 \rangle } + W_{aa} a^{\langle t \rangle } + b)\tag{1}$$

activation:
$$ z^{\langle t + 1 \rangle } = W_{ya}  a^{\langle t + 1 \rangle } + b_y \tag{2}$$

prediction:
$$ \hat{y}^{\langle t+1 \rangle } = softmax(z^{\langle t + 1 \rangle })\tag{3}$$

- $\hat{y}^{\langle t+1 \rangle }$ 세부사항:
   - $\hat{y}^{\langle t+1 \rangle }$는 softmax로 구해진 확률로 이루어져있습니다. (전부 합하면 1입니다)
   - $\hat{y}^{\langle t+1 \rangle}_i$은 확률 값을 갖고 다음 알파벳으로 사용될 알파벳의 인덱스 값을 의미합니다.

#### Additional Hints

- 코드에서 $x^{\langle 1 \rangle}$은 `x`로 표현합니다. 영행렬을 만들때 행 부분은 알파벳 갯수와 같게 설정하고 열 부분은 1로 설정합니다. 이 행렬은 2차원 행렬입니다.
- 코드에서 $a^{\langle 0 \rangle}$는 `a_prev`로 표현합니다. 이것 또한 영행렬입니다. 행 부분은 $n_{a}$값이고 열 부분은 1입니다. 역시 2차원 행렬입니다. ($n_{a}$ 는 $W_{aa}$의 열 부분에서 가져옵니다.)
- [numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html)
- [numpy.tanh](https://docs.scipy.org/doc/numpy/reference/generated/numpy.tanh.html)

#### Using 2D arrays instead of 1D arrays
* 왜 $x^{\langle 1 \rangle}$과 $a^{\langle 0 \rangle}$을 2차원 행렬이라고 강조하는지 궁금하실겁니다.

* numpy로 만들어진 행렬간의 연산을 할때 2차원 행렬과 1차원 벡터 끼리 연산하면 결과로 1차원 행렬이 나오기 때문입니다.

* 행렬간에 합을 할때 두 행렬의 사이즈가 같아야하기 때문에 2차원으로 만드는 겁니다.

* 만약 서로 이차원 행렬이지만 사이즈가 다를때는 파이썬에서 "broadcasts"으로 한 행렬의 사이즈에 맞춰 연산이됩니다.

* 자세한건 아래 샘플 코드를 보고 이해해보세요!

In [ ]:
import numpy as np

In [ ]:
matrix1  = np.array([[1,1],[2,2],[3,3]]) # (3,2)
matrix2  = np.array([[0],[0],[0]])       # (3,1) 
vector1D = np.array([1,1])               # (2,) 
vector2D = np.array([[1],[1]])           # (2,1)
print("matrix1 \n",  matrix1,"\n")
print("matrix2 \n",  matrix2,"\n")
print("vector1D \n", vector1D,"\n")
print("vector2D \n", vector2D)

In [ ]:
print("Multiply 2D and 1D arrays: result is a 1D array\n", 
      np.dot(matrix1,vector1D))
print("Multiply 2D and 2D arrays: result is a 2D array\n", 
      np.dot(matrix1,vector2D))

In [ ]:
print("Adding (3 x 1) vector to a (3 x 1) vector is a (3 x 1) vector\n",
      "This is what we want here!\n", 
      np.dot(matrix1,vector2D) + matrix2)

In [ ]:
print("Adding a (3,) vector to a (3 x 1) vector\n",
      "broadcasts the 1D array across the second dimension\n",
      "Not what we want here!\n",
      np.dot(matrix1,vector1D) + matrix2
     )

- **Step 3**: Sampling: 
    - 이제 $y^{\langle t+1 \rangle}$를 구했으니 공룡이름에 사용할 다음 알파벳을 선택해야합니다. 
    - 만약 가장 높은 확률값을 갖는 알파벳을 선택한다면 모델은 항상 시작점에 같은 알파벳을 선택할겁니다.
        - 결과를 좀 더 흥미롭게 하기 위해 `np.random.choice`를 사용해 첫번째 알파벳을 항상 같은 결과가 나오지 않도록 합니다.
        
    - Sampling은 모든 선택지에서 선택을 하는겁니다.
    - Sampling는 무작위 단어를 만들도록 도와줍니다.
    - $\hat{y}^{\langle t+1 \rangle }$에 따른 확률 분포에 따라 다음 알파벳을 선택합니다.
    - 다른 말로 만약 $\hat{y}^{\langle t+1 \rangle }_i = 0.16$일때 "i" 인덱스를 16% 확률로 뽑았다는 말이됩니다.
    - [np.random.choice](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.choice.html)사용합니다.

    Example of how to use `np.random.choice()`:
    ```python
    np.random.seed(0)
    probs = np.array([0.1, 0.0, 0.7, 0.2])
    idx = np.random.choice([0, 1, 2, 3], p = probs)
    ```
    - 확률 분포에 따라 인덱스를 뽑는다는 의미입니다.

    $P(index = 0) = 0.1, P(index = 1) = 0.0, P(index = 2) = 0.7, P(index = 3) = 0.2$.

    - 확률 값 `p`는 1차원 벡터로 저장합니다.
    - $\hat{y}^{\langle t+1 \rangle}$는 코드에서 `y`라는 변수로 사용되고 2차원 행렬입니다.

##### Additional Hints
- [range](https://docs.python.org/3/library/functions.html#func-range)
- [numpy.ravel](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ravel.html)는 다차원 행렬을 받고 1차원 벡터로 반환합니다.
```Python
arr = np.array([[1,2],[3,4]])
print("arr")
print(arr)
print("arr.ravel()")
print(arr.ravel())
```
Output:
```Python
arr
[[1 2]
 [3 4]]
arr.ravel()
[1 2 3 4]
```

- Note that `append` is an "in-place" operation.  In other words, don't do this:
```Python
fun_hobbies = fun_hobbies.append('learning')  ## Doesn't give you what you want
```

- **Step 4**: Update to $x^{\langle t \rangle }$ 
- 지난 단계에서 `sample()`은 현재시점의 $x^{\langle t \rangle }$ 변수 `x`를 업데이트 해 $x^{\langle t + 1 \rangle }$를 만드는 것입니다.  
- $x^{\langle t + 1 \rangle }$는 이전 예측값에 의해 one-hot 벡터로 만들어집니다.
    
- 순전파를 통해 매 시점에서 $x^{\langle t + 1 \rangle }$를 얻고 반복을 통해 "\n"얻을 때까지 반복해 공룡의 이름을 완성시킵니다.

##### Additional Hints
- 시작할때 `x`값을 초기화 하기 위해 영 행렬로 만듭니다.
    - 영행렬을 만들때: [numpy.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html)
    - 또는 행렬을 한가지 값으로 채울때: [numpy.ndarray.fill](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.fill.html)

In [ ]:
# GRADED FUNCTION: sample
def sample(parameters, char_to_ix, seed):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN

    Arguments:
    parameters -- python dictionary containing the parameters Waa, Wax, Wya, by, and b. 
    char_to_ix -- python dictionary mapping each character to an index.
    seed -- used for grading purposes. Do not worry about it.

    Returns:
    indices -- a list of length n containing the indices of the sampled characters.
    """
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a        = Waa.shape[1]
    
    # Step 1: Create the a zero vector x that can be used as the one-hot vector 
    # representing the first character (initializing the sequence generation). (≈1 line)
    x      = np.zeros((vocab_size,1))
    # Step 1': Initialize a_prev as zeros (≈1 line)
    a_prev = np.zeros((n_a, 1))
    
    # Create an empty list of indices, this is the list which will contain the list of indices of the characters to generate (≈1 line)
    indices = []
    
    # idx is the index of the one-hot vector x that is set to 1
    # All other positions in x are zero.
    # We will initialize idx to -1
    idx = -1 
    
    # Loop over time-steps t. At each time-step:
    # sample a character from a probability distribution 
    # and append its index (`idx`) to the list "indices". 
    # We'll stop if we reach 50 characters 
    # (which should be very unlikely with a well trained model).
    # Setting the maximum number of characters helps with debugging and prevents infinite loops. 
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        # Step 2: Forward propagate x using the equations (1), (2) and (3)
        a = np.tanh( np.dot(Wax, x) + np.dot(Waa, a_prev) + b )
        z = np.dot( Wya, a ) + by
        y = softmax( z )
        
        np.random.seed( counter + seed ) 
        
        # Step 3: Sample the index of a character within the vocabulary from the probability distribution y
        idx = np.random.choice(vocab_size, 1, p=y.flatten())
        
        # Append the index to "indices"
        indices.append( idx[0] )
        
        # Step 4: Overwrite the input x with one that corresponds to the sampled index `idx`.
        x      = np.zeros((vocab_size,1))
        x[idx] = 1
        
        # Update "a_prev" to be "a"
        a_prev = a
        
        # for grading purposes
        seed    += 1
        counter +=1

    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [ ]:
np.random.seed(2)
_, n_a = 20, 100
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)

parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}

indices = sample(parameters, char_to_ix, 0)
print("Sampling:")
print("list of sampled indices:\n", indices)
print("list of sampled characters:\n", [ix_to_char[i] for i in indices])

** Expected output:**

```Python
Sampling:
list of sampled indices:
[12, 17, 24, 14, 13, 9, 10, 22, 24, 6, 13, 11, 12, 6, 21, 15, 21, 14, 3, 2, 1, 21, 18, 24, 7, 25, 6, 25, 18, 10, 16, 2, 3, 8, 15, 12, 11, 7, 1, 12, 10, 2, 7, 7, 11, 17, 24, 12, 13, 24, 0]
list of sampled characters:
['l', 'q', 'x', 'n', 'm', 'i', 'j', 'v', 'x', 'f', 'm', 'k', 'l', 'f', 'u', 'o', 'u', 'n', 'c', 'b', 'a', 'u', 'r', 'x', 'g', 'y', 'f', 'y', 'r', 'j', 'p', 'b', 'c', 'h', 'o', 'l', 'k', 'g', 'a', 'l', 'j', 'b', 'g', 'g', 'k', 'q', 'x', 'l', 'm', 'x', '\n']
```

## 3 - Building the language model 

이제 단어 생성기 모델을 만들어봅시다.

### 3.1 - Gradient descent 

* 학습데이터 한개씩 한 시점에 사용할겁니다. 따라서 최적화 알고리즘으로 SGD를 사용합니다. (clipped gradients와 함께)


RNN 에서 최적화하는 방법을 다시한번 상기시켜봅시다.
- RNN 순전파를 통해 손실함수 계산
- 역전파를 통해 모든 매개변수의 변화량을 계산
- 변화량이 너무 커지거나 너무 작아지지 않도록 Clip 적용
- 경사하강법을 사용해 매개변수 업데이트

**Exercise**: SGD 최적화 기법을 구현해보세요~

아래 함수들은 제공됩니다:
```python
def rnn_forward(X, Y, a_prev, parameters):
    """ Performs the forward propagation through the RNN and computes the cross-entropy loss.
    It returns the loss' value as well as a "cache" storing values to be used in backpropagation."""
    ....
    return loss, cache
    
def rnn_backward(X, Y, parameters, cache):
    """ Performs the backward propagation through time to compute the gradients of the loss with respect
    to the parameters. It returns also all the hidden states."""
    ...
    return gradients, a

def update_parameters(parameters, gradients, learning_rate):
    """ Updates parameters using the Gradient Descent Update Rule."""
    ...
    return parameters
```

그리고 위에서 만든 `clip`함수를 사용합니다:

```Python
def clip(gradients, maxValue)
    """Clips the gradients' values between minimum and maximum."""
    ...
    return gradients
```

In [ ]:
# GRADED FUNCTION: optimize

def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    Execute one step of the optimization to train the model.
    
    Arguments:
    X -- list of integers, where each integer is a number that maps to a character in the vocabulary.
    Y -- list of integers, exactly the same as X but shifted one index to the left.
    a_prev -- previous hidden state.
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    learning_rate -- learning rate for the model.
    
    Returns:
    loss -- value of the loss function (cross-entropy)
    gradients -- python dictionary containing:
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dWya -- Gradients of hidden-to-output weights, of shape (n_y, n_a)
                        db -- Gradients of bias vector, of shape (n_a, 1)
                        dby -- Gradients of output bias vector, of shape (n_y, 1)
    a[len(X)-1] -- the last hidden state, of shape (n_a, 1)
    """
    # Forward propagate through time (≈1 line)
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    # Backpropagate through time (≈1 line)
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    # Clip your gradients between -5 (min) and 5 (max) (≈1 line)
    gradients = clip(gradients, 5)
    
    # Update parameters (≈1 line)
#     parameters = update_parameters(parameters, gradients, learning_rate)    
    for key in parameters.keys():
        parameters[key] = parameters[key] - gradients['d'+key] * learning_rate 
    
    return loss, gradients, a[len(X)-1]

In [ ]:
np.random.seed(1)
vocab_size, n_a = 27, 100
a_prev = np.random.randn(n_a, 1)
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}
X = [12,3,5,11,22,3]
Y = [4,14,11,22,25, 26]

loss, gradients, a_last = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
print("Loss =", loss)
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])
print("np.argmax(gradients[\"dWax\"]) =", np.argmax(gradients["dWax"]))
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])
print("gradients[\"db\"][4] =", gradients["db"][4])
print("gradients[\"dby\"][1] =", gradients["dby"][1])
print("a_last[4] =", a_last[4])

** Expected output:**

```Python
Loss = 126.503975722
gradients["dWaa"][1][2] = 0.194709315347
np.argmax(gradients["dWax"]) = 93
gradients["dWya"][1][2] = -0.007773876032
gradients["db"][4] = [-0.06809825]
gradients["dby"][1] = [ 0.01538192]
a_last[4] = [-1.]
```

### 3.2 - Training the model 

* 학습용으로 주어진 공룡이름 데이터의 각 줄(이름 1개)을 학습데이터 1개로 사용합니다.
* SGD를 100번 거칠때 마다 10개 예측값을 만들어 모델이 잘 작동되는지 확인 할 수 있도록 만들겁니다.
* 데이터셋을 랜덤하게 섞어 SGD로 학습할때 랜덤한 이름으로 학습되는것을 기억하세요~


**Exercise**: 
구현할 `model()`에서 
Follow the instructions and implement `model()`. When `examples[index]` contains one dinosaur name (string), to create an example (X, Y), you can use this:

##### Set the index `idx` into the list of examples
* "examples"라는 변수명을 한 리스트는 무작위 순서로 공룡이름을 저장하고 있고 반복문에 사용됩니다.

* 만약 "examples"에 100개 데이터가 있고 반복문을 적용했을때 100번 이후 추가로 계속 데이터에 접근 하려면 어떻게 해야할까요?

* Hint: 101 나누기 100 은 0입니다. (나머지 값 1)
* `%`은 나머지를 구하는 연산자입니다.

##### Extract a single example from the list of examples
* `single_example`: 인덱스 정보인 `idx`를 사용해 전에 얻은 알파벳을 알 수 있습니다.

##### Convert a string into a list of characters: `single_example_chars`
* `single_example_chars`: 한 str 객체는 알파벳 또는 문자들을 list 집합입니다.

* 반복문을 사용해 str객체를 할 문자 단위로 나눠 list 자료형으로 바꿀 수 있습니다.
```Python
str = 'I love learning'
list_of_chars = [c for c in str]
print(list_of_chars)
```

```
['I', ' ', 'l', 'o', 'v', 'e', ' ', 'l', 'e', 'a', 'r', 'n', 'i', 'n', 'g']
```

##### Convert list of characters to a list of integers: `single_example_ix`
* 각 알파벳에 연관된 인덱스 숫자를 저장한 리스트를 만들어봅시다.
* 자료형 dict인 `char_to_ix`를 사용합니다.
* 특정 string 객체에서 추출한 알파벳 리스트 객체와 `char_to_ix`를 연관지어 만듭니다.

##### Create the list of input characters: `X`
* `rnn_forward`에서 `None`값을 사용합니다. 입력값을 zero-vector로 하기 위해 None을 사용합니다.
* 입력값인 알파벳들이 저장된 리스트 앞부분에 `None`을 추가합니다.
* 리스트에 값을 추가하는 것은 다양한 방법이 있습니다. 이번에 사용할 방법은 `['a'] + ['b']`식으로 두 리스트를 하나로 합치는 겁니다.

##### Get the integer representation of the newline character `ix_newline`
* `ix_newline`: 공룡 이름의 끝을 표현하기 위해 `'\n'`를 사용합니다.
    - 따라서 `'\n'`의 인덱스 값이 필요합니다.
    - `char_to_ix`를 사용하세요~

##### Set the list of labels (integer representation of the characters): `Y`
* RNN 학습의 목적은 이름의 다음 알파벳을 예측하는 겁니다. 따라서 $t$시점의 입력값 X와 매핑되는 Y값은 $t-1$시점 Y값입니다.
    - 예를들어, `Y[0]` 는 `X[1]`과 같은 값입니다.
* 또한 RNN 모델은 공룡이름의 끝을 의미하는 newline 문자 또한 예측해야합니다.

In [ ]:
# GRADED FUNCTION: model
def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):
    """
    Trains the model and generates dinosaur names. 
    Arguments:
    data -- text corpus
    ix_to_char -- dictionary that maps the index to a character
    char_to_ix -- dictionary that maps a character to an index
    num_iterations -- number of iterations to train the model for
    n_a -- number of units of the RNN cell
    dino_names -- number of dinosaur names you want to sample at each iteration. 
    vocab_size -- number of unique characters found in the text (size of the vocabulary)
    Returns:
    parameters -- learned parameters
    """
    # Retrieve n_x and n_y from vocab_size
    n_x, n_y = vocab_size, vocab_size
    
    # Initialize parameters
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    # Initialize loss (this is required because we want to smooth our loss)
    loss = get_initial_loss(vocab_size, dino_names)
    
    # Build list of all dinosaur names (training examples).
    with open("dinos.txt") as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
    # Shuffle list of all dinosaur names
    np.random.seed(0)
    np.random.shuffle(examples)
    
    # Initialize the hidden state of your LSTM
    a_prev = np.zeros((n_a, 1))
    
    # Optimization loop
    for j in range(num_iterations):

        # Set the index `idx` (see instructions above)
        idx = j % len(examples)
        
        # Set the input X (see instructions above)
        single_example       = examples[idx]
        single_example_chars = list(single_example)
        single_example_ix    = [ char_to_ix[char] for char in single_example_chars ]
        X                    = [None] + single_example_ix
        
        # Set the labels Y (see instructions above)
        ix_newline = char_to_ix['\n']
        Y          = X[1:] + [ix_newline]
        
        # Perform one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
        # Choose a learning rate of 0.01
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)
        
        # Use a latency trick to keep the loss smooth. It happens here to accelerate the training.
        loss = smooth(loss, curr_loss)

        # Every 2000 Iteration, generate "n" characters thanks to sample() to check if the model is learning properly
        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            # The number of dinosaur names to print
            seed = 0
            for name in range(dino_names):
                
                # Sample indices and print them
                sampled_indices = sample(parameters, char_to_ix, seed)
                print_sample(sampled_indices, ix_to_char)
                
                seed += 1  # To get the same result (for grading purposes), increment the seed by one. 
      
            print('\n')
        
    return parameters

아래 셀을 실행시켜보면 학습을 하면서 2000번 학습할때마다 학습한 내용을 기반으로 공룡이름을 창조해냅니다. 학습 횟수가 늘어날 수록 점점 얼추 공룡이름과 비슷하게 만들어 집니다.

In [ ]:
parameters = model(data, ix_to_char, char_to_ix)

** Expected Output**

The output of your model may look different, but it will look something like this:

```Python
Iteration: 34000, Loss: 22.447230

Onyxipaledisons
Kiabaeropa
Lussiamang
Pacaeptabalsaurus
Xosalong
Eiacoteg
Troia
```

## Conclusion

You can see that your algorithm has started to generate plausible dinosaur names towards the end of the training. At first, it was generating random characters, but towards the end you could see dinosaur names with cool endings. Feel free to run the algorithm even longer and play with hyperparameters to see if you can get even better results. Our implementation generated some really cool names like `maconucon`, `marloralus` and `macingsersaurus`. Your model hopefully also learned that dinosaur names tend to end in `saurus`, `don`, `aura`, `tor`, etc.

If your model generates some non-cool names, don't blame the model entirely--not all actual dinosaur names sound cool. (For example, `dromaeosauroides` is an actual dinosaur name and is in the training set.) But this model should give you a set of candidates from which you can pick the coolest! 

This assignment had used a relatively small dataset, so that you could train an RNN quickly on a CPU. Training a model of the english language requires a much bigger dataset, and usually needs much more computation, and could run for many hours on GPUs. We ran our dinosaur name for quite some time, and so far our favorite name is the great, undefeatable, and fierce: Mangosaurus!

<img src="images/mangosaurus.jpeg" style="width:250;height:300px;">

## 4 - Writing like Shakespeare

유사하지만 좀 더 복잡한 프로젝트로 LSTM을 사용해 셰익스피어 시를 만들어보겠습니다.

공룡이름으로 학습한 대신 셰익스피어 시들을 학습합니다.

LSTM cell을 사용하면 단어가 멀리 있어도 영향을 미치는 관계에 대해 학습 시킬 수 있습니다.


<img src="images/shakespeare.jpg" style="width:500;height:400px;">
<caption><center> Let's become poets! </center></caption>

Keras를 사용해 셰익스피어 시를 만들어 보겠습니다.

아래 코드를 실행시켜 모델에 필요한 패키지를 호출합니다. 몇분 소요될 수 있습니다.


In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from shakespeare_utils import *
import sys
import io

당신의 시간을 아끼기 위해 셰익스피어 시집 [*"The Sonnets"*](shakespeare.txt)으로 1000epoch 정도 학습한 모델이 있습니다.

미리 학습된 모델에 조금 더 학습시켜보겟습니다.

학습이 끝나면 `generate_output`를 실행시킬 수 있게되는데 입력값을 직접 입력해야합니다. (입력값: 40글자 이하 문장)

그럼 입력값으로 주어진 문장으로 시가 시작되고 모델이 시의 끝까지 작성합니다.

에를들어 "Forsooth this maketh no sense "를 입력해보세요~ (따옴표는 넣지 않습니다!!)

입력값의 마지막에 띄어쓰기 유무에 따라 결과가 달라지니 두개 모두 확인해보세요~


In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y, batch_size=128, epochs=1, callbacks=[print_callback])

In [ ]:
# Run this cell to try with different inputs without having to re-train the model 
generate_output()

The RNN-Shakespeare model is very similar to the one you have built for dinosaur names. 
RNN-Shakespeare은 공룡이름 생성기와 거의 비슷합니다. 다른점은..
- 기본 RNN 구조가 아닌 LSTM 구조를 사용해 먼 거리의 관계 까지 학습시켰습니다.
- 그리고 LSTM 모델은 더 깊습니다.
- Keras API를 사용했습니다.

더 궁금한 점이 있다면 https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py 에 방문해보세요~

**References**:
- This exercise took inspiration from Andrej Karpathy's implementation: https://gist.github.com/karpathy/d4dee566867f8291f086. To learn more about text generation, also check out Karpathy's [blog post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/).
- For the Shakespearian poem generator, our implementation was based on the implementation of an LSTM text generator by the Keras team: https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py 